In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt

import pandas as pd
from learners import *
from dataset import *
from explanations import *

In [ ]:
from responsibly.dataset import AdultDataset

In [ ]:
df = AdultDataset().df
y = np.array(df['income_per_year'] == '>50K')

In [ ]:
categorical_cols = ['race', 'sex', 'workclass', 'education', 
                    'marital_status', 'occupation', 'relationship', 'native_country']
cont_cols = ['age', 'education-num', 'capital_gain', 'capital_loss', 'hours_per_week']

In [ ]:
discrete_df = pd.concat([pd.get_dummies(df[cat]).add_prefix(cat + "_") for cat in categorical_cols], axis=1)
cont_df = df[cont_cols]
new_df = pd.concat([discrete_df, cont_df], axis=1)

In [ ]:
final_cols = new_df.columns

In [ ]:
final_cols

In [ ]:
X = new_df.to_numpy()

In [ ]:
sensitive_X = X[:, :7]
non_sensitive_X = X[:, 7:]

In [ ]:
'''
Standardize features
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy = True, with_mean = True, with_std = True)
scaler.fit(non_sensitive_X)
non_sensitive_X = scaler.transform(non_sensitive_X)

In [ ]:
X = np.hstack([sensitive_X, non_sensitive_X])

In [ ]:
w_star = train_logistic_regression(X, y)

In [ ]:
X1 = append_offset(X)
evaluate_linear(w_star, X1, y)

In [ ]:
NUM_IMM_FEATS = 7

In [ ]:
NUM_RACE_FEAT = 4

In [ ]:
DR = "latest_figs"

## Get Margin Points

In [ ]:
def measure_gap(pt1, pt2, thres=0.03):
    mutable_feat1 = pt1[NUM_IMM_FEATS:] #non mutable features only
    mutable_feat1 = pt2[NUM_IMM_FEATS:]
    edit_dist = np.linalg.norm(mutable_feat1 - mutable_feat1)
    return edit_dist  <= thres

In [ ]:
def get_margin_points(X, w, norm_thres=0.03, r=0.03, pairs=True):
    neg_candidate_idx, pos_candidate_idx = find_linear_margin(X, w, r=norm_thres)
    
    margin_idx = set()
    reachable = []
    for i in neg_candidate_idx:
        for j in pos_candidate_idx:
            pt1, pt2 = X[i], X[j]
            if measure_gap(pt1, pt2, thres=r):
                reachable.append((pt1, pt2))
                margin_idx.add(i)
                margin_idx.add(j)
    print("number of margin points {}".format(len(margin_idx)))
    
    if len(margin_idx) == 0: return np.array([]), X
    
    non_margin_idx = list(set(range(len(X))).difference(margin_idx))
    non_margin_pts = X[non_margin_idx]

    margin_idx = np.array(list(margin_idx))
    margin_pts = X[margin_idx]
    
    if pairs:
        return reachable
    
    return margin_pts, non_margin_pts

## Is SVM better than logistic regression

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC(random_state=0).fit(X, y)

In [ ]:
svm.score(X, y)

In [ ]:
svm_w = svm.coef_.reshape(-1)
svm_w = np.append(svm_w, svm.intercept_)
evaluate_linear(svm_w, X1, y)

In [ ]:
lr_svm_nums = []
rs = [0.01, 0.02, 0.03, 0.05, 0.08, 0.1]
for r in rs:
    margin_pairs = get_margin_points(X1, w_star, norm_thres=r, r=r, pairs=True)
    svm_pairs = get_margin_points(X1, svm_w, norm_thres=r, r=r, pairs=True)
    lr_svm_nums.append((len(margin_pairs), len(svm_pairs)))

In [ ]:
lr_color = '#cc004c'
svm_color = '#0089d0'
X_color = '#f37021'

In [ ]:
G = rs
X_axis = np.arange(len(G))
  
plt.bar(X_axis - 0.2, [x[0] for x in lr_svm_nums], 0.4, label = 'LR margin size', color=lr_color)
plt.bar(X_axis + 0.2, [x[1] for x in lr_svm_nums], 0.4, label='SVM margin size', color=svm_color)

plt.xticks(X_axis, G)
plt.xlabel("r")
plt.ylabel("Number of margin pairs")
plt.legend()
plt.tight_layout()

plt.savefig(DR + "/margin_lr_svm.pdf", box_inches = 'tight', pad_inches = 0)
plt.close()

## Does SVM vs LR change composition of margin?

In [ ]:
final_cols[:NUM_IMM_FEATS]

In [ ]:
margin_pts, non_margin_pts = get_margin_points(X1, w_star, norm_thres=0.03, r=0.03, pairs=False)

In [ ]:
'''
Logistic regression
'''
G = ['American Indian','AAPI','Black','Other', 'White']
X_axis = np.arange(len(G))
  
plt.bar(X_axis - 0.2, 100 * np.average(margin_pts[:, :len(G)], axis=0), 0.4, label = 'Margin Rate', color=lr_color)
plt.bar(X_axis + 0.2, 100 * np.average(X[:, :len(G)], axis=0), 0.4, label='Base Rate', color=X_color)
  
plt.xticks(X_axis, G)
plt.xlabel("Groups")
plt.ylabel("Percentage")
plt.legend()
plt.tight_layout()

plt.savefig(DR + "/margin_rate_lr.pdf", box_inches = 'tight', pad_inches = 0)
plt.close()

In [ ]:
np.average(margin_pts[:, :len(G)], axis=0)

In [ ]:
'''
Very similar make up: X vs non-margin
'''
np.average(non_margin_pts[:, :len(G)], axis=0), np.average(X[:, :len(G)], axis=0)

In [ ]:
'''
SVM does not change the number of margin points by much
'''
svm_margin_pts, svm_non_margin_pts = get_margin_points(X1, svm_w, norm_thres=0.03, r=0.03, pairs=False)

In [ ]:
G = ['American Indian','AAPI','Black','Other', 'White']
X_axis = np.arange(len(G))
  
plt.bar(X_axis - 0.2, 100 * np.average(svm_margin_pts[:, :len(G)], axis=0), 0.4, 
        label = 'Margin Rate', color=svm_color)
plt.bar(X_axis + 0.2, 100 * np.average(X[:, :len(G)], axis=0), 0.4, label='Base Rate', color=X_color)
  
plt.xticks(X_axis, G)
plt.xlabel("Groups")
plt.ylabel("Percentage")
plt.legend()
plt.tight_layout()

plt.savefig(DR + "/margin_rate_svm.pdf", box_inches = 'tight', pad_inches = 0)
plt.close()

In [ ]:
np.average(svm_margin_pts[:, :len(G)], axis=0)

## Deletion of margin and then assessing DP fairness after deletion

In [ ]:
def dp(X, feat_no, w):
    group_X = X[X[:, feat_no] == 1]
    y_hat = group_X @ w >= 0
    return np.average(y_hat)

In [ ]:
margin_pts, non_margin_pts = get_margin_points(X1, w_star, norm_thres=0.03, r=0.03, pairs=False)

In [ ]:
'''
Compare DP assessed on non-margin vs X
'''
[dp(non_margin_pts, i, w_star) for i in range(NUM_RACE_FEAT)], [dp(X1, i, w_star) for i in range(NUM_RACE_FEAT)]